# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [1]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = 'c2oX5gf7C6ADXTKnm0qeje33C1aXCjaHg11kjzyz	'  # Please replace by your own Freesound API key
FILES_DIR = 'amplab-freesound-master'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags', 'ac_analysis']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [2]:
# Define some util functions

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        #filter += ' ac_tonality_confidence:[0.9 TO 1.0] '
        #filter += ' ac_tonality:"A major" '
        filter += ' duration:[0 TO 60] '  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    
    if sound.as_dict()['ac_analysis'] is not None:
    
        record['ac_tonality'] = sound.as_dict()['ac_analysis']['ac_tonality']
    
    else:
        
         record['ac_tonality'] = None 
    
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record
    print (record)

In [3]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    {
        'query': 'african vocals',
        'filter': 'duration:[0 TO 10] ac_tonality:"A major" ac_tonality_confidence:[0.8 TO 1.0]',
        'num_results': 10, 
    },
    {
        'query': 'drums',
        'filter': 'duration:[0 TO 1]',
        'num_results': 20, 
    },
    {
        'query': 'pads',
        'filter': 'duration:[0 TO 15] ac_tonality:"A major" ac_tonality_confidence:[0.8 TO 1]', 
        'num_results': 30, 
    },
]


# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 50 entries! dataframe.csv


,ac_analysis,ac_tonality,freesound_id,license,name,path,tags,username
0,"{'ac_tempo_confidence': 0.0, 'ac_note_confiden...",G# minor,384671,http://creativecommons.org/publicdomain/zero/1.0/,Drum R 1.wav,files/384671_5591103-hq.ogg,"[drums, tamil, Indian]",mathan24
1,None,None,459895,http://creativecommons.org/publicdomain/zero/1.0/,SH01a Bass Drum 004,files/459895_4448255-hq.ogg,"[bassdrum, kick, bd, drums, percussion, bass-d...",ANARKYA
2,"{'ac_tempo_confidence': 0.0, 'ac_note_confiden...",A minor,118524,http://creativecommons.org/publicdomain/zero/1.0/,m3mphiz_drum_1.wav,files/118524_1753199-hq.ogg,"[bass, drums, hard, kick]",M3MPHiZ
3,"{'ac_tempo_confidence': 0.0, 'ac_note_confiden...",D major,400707,http://creativecommons.org/publicdomain/zero/1.0/,Subby Kick Drum,files/400707_1126957-hq.ogg,"[loop, groove, kickdrum, techno, garage, bpm, ...",Mattc90
4,"{'ac_tempo_confidence': 0.0, 'ac_note_confiden...",C# major,269716,http://creativecommons.org/publicdomain/zero/1.0/,Electronic Kick Drum #1,files/269716_4965320-hq.ogg,"[Synthesizer, Music, Closed, Digital, Electro,...",IanStarGem
5,"{'ac_tempo_confidence': 0.0, 'ac_note_confiden...",B minor,270276,http://creativecommons.org/publicdomain/zero/1.0/,Electronic Snare Drum #2,files/270276_4965320-hq.ogg,"[Edison, Punchy, Sytrus, Synthesizer, Music, E...",IanStarGem
6,"{'ac_tempo_confidence': 0.0, 'ac_note_confiden...",D major,318331,http://creativecommons.org/licenses/by/3.0/,Snare Drums Oneshot (Lord Lokus),files/318331_1990402-hq.ogg,"[Drums, Beat, Oneshot, Samples, Snare, SD]",Freakshitor669
7,"{'ac_tempo_confidence': 0.0, 'ac_note_confiden...",B major,198104,http://creativecommons.org/licenses/by/3.0/,snare,files/198104_3680406-hq.ogg,"[kick, music, bass, drums, edm]",owowowwo
8,"{'ac_tempo_confidence': 0.0, 'ac_note_confiden...",F# minor,410149,http://creativecommons.org/publicdomain/zero/1.0/,Kick Drum,files/410149_7383104-hq.ogg,"[bass, Kick, drum, drums, trap, music, future,...",ScreamStudio
9,None,None,469656,http://creativecommons.org/publicdomain/zero/1.0/,Simmons Drums (12).wav,files/469656_5876986-hq.ogg,"[Electronic-Drums, Simmons, Percussion, Simmon...",tarane468
